In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["NUM_THREADS"] = "8"
os.environ["OMP_NUM_THREADS"] = "8"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs  
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping


# Visualize the result
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


#bands = [6, 7, 20, 28, 28, 31]
bands=[29]
#bands=[1]

total cores: 256
1 Physical GPUs, 1 Logical GPUs


2024-03-14 13:35:37.519700: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2024-03-14 13:35:37.519917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21321 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [62]:
import extract_training_data 
importlib.reload(extract_training_data)
from extract_training_data import *
folder = "/uio/hume/student-u37/fslippe/for_Rob/"
start = "20200303"
end = "20230306"
start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
x, dates, masks, lon_lats, times = extract_1km_data(folder,
                                                         bands=[1,2],
                                                         start_date=start_converted,
                                                         end_date=end_converted,
                                                         return_lon_lat=True,
                                                         data_type="hdf",
                                                         combine_pics=True)

total cores: 256
['/uio/hume/student-u37/fslippe/for_Rob/']
2023055   930
2023055   935
2023055   940
2023055   1110
2023055   1115


  0%|          | 0/1 [00:00<?, ?it/s]

EV_250_Aggr1km_RefSB
EV_250_Aggr1km_RefSB
EV_250_Aggr1km_RefSB
EV_250_Aggr1km_RefSB
EV_250_Aggr1km_RefSB


100%|██████████| 1/1 [00:07<00:00,  7.44s/it]

2
2


In [74]:

def create_nc_from_extracted_data(x, dates, masks, lon_lats, times, save_folder):
    from datetime import datetime, timedelta

    def parse_date(date_str, time_str):
        # Parse the date string 'YYYYddd'
        year = int(date_str[:4])
        day_of_year = int(date_str[4:])
        
        # Convert the day of the year to month and day
        date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)
        
        # Parse the time string 'HHMM' and create a time delta
        hours = int(time_str[:-2])
        minutes = int(time_str[-2:])
        
        # Add the time delta to the date
        date_time = date + timedelta(hours=hours, minutes=minutes)
        
        return date_time

    # Create an empty list to hold your datasets
    datasets = []
    lon = [lon_lats[0][0, :, :], lon_lats[1][0, :, :]]
    lat = [lon_lats[0][1, :, :], lon_lats[1][1, :, :]]
    for i, (x_array, date, time) in enumerate(zip(x, dates, times)):
        # Parse the date and time using the `parse_date` function
        date_time = parse_date(str(date), str(time).zfill(4))  # Assuming date & time are integers

        # Create a DataArray for 'x' and add 'time' dimension and coordinate
        x_da = xr.DataArray(
            data=x_array[None, ...], 
            dims=('time', 'x', 'y', 'band'),  
            coords={'time': [date_time]}      
        )

        # Create the Dataset for the current time step
        ds = xr.Dataset(
            data_vars={'radiance': x_da},
            coords={'time': [date_time]}  
        )
        
        # Add 'masks', 'lon', and 'lat' to the Dataset for the current time step
        ds['masks'] = xr.DataArray(masks[i][None, ...], dims=('time', 'x', 'y'), coords={'time': [date_time]})
        ds['lon'] = xr.DataArray(lon[i][None, ...], dims=('time', 'x', 'y'), coords={'time': [date_time]})
        ds['lat'] = xr.DataArray(lat[i][None, ...], dims=('time', 'x', 'y'), coords={'time': [date_time]})
        
        # And do the same for any other variables, such as 'mod_min'
        
        # Append the dataset to the list of datasets
        ds.to_netcdf(save_folder + f"MOD021KM.A{date}.{str(time).zfill(4)}.nc")

        
create_nc_from_extracted_data(x, dates, masks, lon_lats, times, folder)

In [75]:
ds

<xarray.Dataset>
Dimensions:   (time: 1, x: 2716, y: 1076, band: 2)
Coordinates:
  * time      (time) datetime64[ns] 2023-02-24T11:10:00
Dimensions without coordinates: x, y, band
Data variables:
    radiance  (time, x, y, band) float32 0.3022 0.2693 0.272 ... 61.8 41.45
    masks     (time, x, y) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    lon       (time, x, y) float32 -0.3858 -0.2087 -0.0332 ... 7.19 7.223 7.257
    lat       (time, x, y) float32 83.73 83.72 83.72 83.71 ... 56.44 56.43 56.42